# Imports

In [ ]:
!pip install -U clifford

In [ ]:
import h5py
import gdown
import numpy as np
from clifford import Cl
from tqdm import tqdm
import pickle

# Dataset and multivectors

### The dataset contains 1999 bifurcating samples and 1999 single samples

In [ ]:
layout, blades = Cl(3, 0, 1) # geometric algebra G(3,0,1)

In [ ]:
blades # these are the 16 orthogonal basis of G(3,0,1)

{'': 1,
 'e1': (1^e1),
 'e2': (1^e2),
 'e3': (1^e3),
 'e4': (1^e4),
 'e12': (1^e12),
 'e13': (1^e13),
 'e14': (1^e14),
 'e23': (1^e23),
 'e24': (1^e24),
 'e34': (1^e34),
 'e123': (1^e123),
 'e124': (1^e124),
 'e134': (1^e134),
 'e234': (1^e234),
 'e1234': (1^e1234)}

## Bifurcating

In [ ]:
# get the dataset from Gdrive
bifurcating_dataset_file_id = '1oGqW1M2B6AY389ZKIvkDm8RpZcOZQdC0'
download_link = f'https://drive.google.com/uc?id={bifurcating_dataset_file_id}'
bifurcating_samples = gdown.download(download_link, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1oGqW1M2B6AY389ZKIvkDm8RpZcOZQdC0
To: /content/database.hdf5
100%|██████████| 1.76G/1.76G [00:14<00:00, 119MB/s]


In [ ]:
bif_samples = [] # list of all the bifurcating samples
damaged_bif_samples = [] # list of indeces of damaged bif samples
bif_multivectors = [] # list of bif multivectors

with h5py.File(bifurcating_samples, 'r') as hdf5_file:

  for i in tqdm(range(1999)):
    # to set the correct sample index as a string
    if i < 10:
      bif_sample_idx = 'sample_000' + str(i)
    elif i < 100:
      bif_sample_idx = 'sample_00' + str(i)
    elif i < 1000:
      bif_sample_idx = 'sample_0' + str(i)
    else:
      bif_sample_idx = 'sample_' + str(i)

    try:
      bif_sample = hdf5_file[bif_sample_idx]
      bif_samples.append(bif_sample)

      # wss as scalar
      wss_multivector = 0
      for row in bif_sample['wss'][()]:
        vector = row[0] * blades[''] + row[1] * blades[''] + row[2] * blades['']
        wss_multivector += vector

      # pressure as a pseudoscalar
      pressure_multivector = 0
      for elem in bif_sample['pressure'][()]:
        vector = elem * blades['e1234']
        pressure_multivector += vector

      # pos as a point
      pos_multivector = 0
      for row in bif_sample['pos'][()]:
        vector = row[0] * blades['e123'] + row[1] * blades['e124'] + row[2] * blades['e134']
        pos_multivector += vector

      # face as a plane
      face_multivector = 0
      for row in bif_sample['face'][()]:
        vector = row[0] * blades['e1'] + row[1] * blades['e2'] + row[2] * blades['e3']
        face_multivector += vector

      # inlet as a scalar
      inlet_multivector = 0
      for elem in bif_sample['inlet_idcs'][()]:
        vector = elem * blades['']
        inlet_multivector += vector


      total_multivector = wss_multivector + pressure_multivector + pos_multivector + face_multivector + inlet_multivector
      bif_multivectors.append(total_multivector)

    except:
      damaged_bif_samples.append(bif_sample_idx)


  3%|▎         | 66/1999 [02:58<57:54,  1.80s/it]

In [ ]:
# to save the bif_multivectors array in a pickle file on gdrive

# Specify the path on Google Drive
bif_pickle_file_path = '/content/drive/MyDrive/Deep_Learning/Project/bif_multivectors.pkl'

# Save the array to a pickle file on Google Drive
with open(bif_pickle_file_path, 'wb') as pickle_file:
    pickle.dump(bif_multivectors, pickle_file)

In [ ]:
print('number of damaged bif samples:\n', len(damaged_bif_samples))

number of damaged bif samples:
 1


In [ ]:
print('number of correct bif samples:\n', len(bif_samples))

number of correct bif samples:
 1998


In [ ]:
print('number of bif multivectors:\n', len(bif_multivectors))

In [ ]:
print('dimension of each sin multivector:\n', max(bif_multivectors[1].grades())) # if 4,  then the dimension is correct

# Single

In [ ]:
# get the dataset from Gdrive
single_dataset_file_id = '13yLm1UaTPHh7VlZlHtw4giCXp1jARHYd'
download_link = f'https://drive.google.com/uc?id={single_dataset_file_id}'
single_samples = gdown.download(download_link, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=13yLm1UaTPHh7VlZlHtw4giCXp1jARHYd
To: /content/database.hdf5
100%|██████████| 966M/966M [00:10<00:00, 94.7MB/s]


In [ ]:
sin_samples = [] # list of all the single samples
damaged_sin_samples = [] # list of indeces of damaged sin samples
sin_multivectors = [] # list of sin multivectors

with h5py.File(single_samples, 'r') as hdf5_file:

  for i in tqdm(range(1999)):
    # to set the correct sample index as a string
    if i < 10:
      sin_sample_idx = 'sample_000' + str(i)
    elif i < 100:
      sin_sample_idx = 'sample_00' + str(i)
    elif i < 1000:
      sin_sample_idx = 'sample_0' + str(i)
    else:
      sin_sample_idx = 'sample_' + str(i)

    try:
      sin_sample = hdf5_file[sin_sample_idx]
      sin_samples.append(sin_sample)

      # wss as a scalar
      wss_multivector = 0
      for row in sin_sample['wss'][()]:
        vector = row[0] * blades[''] + row[1] * blades[''] + row[2] * blades['']
        wss_multivector += vector

      # pressure as a pseudoscalar
      pressure_multivector = 0
      for elem in sin_sample['pressure'][()]:
        vector = elem * blades['e1234']
        pressure_multivector += vector

      # pos as a point
      pos_multivector = 0
      for row in sin_sample['pos'][()]:
        vector = row[0] * blades['e123'] + row[1] * blades['e124'] + row[2] * blades['e134']
        pos_multivector += vector

      # face as a plane
      face_multivector = 0
      for row in sin_sample['face'][()]:
        vector = row[0] * blades['e1'] + row[1] * blades['e2'] + row[2] * blades['e3']
        face_multivector += vector

      # inlet as a scalar
      inlet_multivector = 0
      for elem in sin_sample['inlet_idcs'][()]:
        vector = elem * blades['']
        inlet_multivector += vector


      total_multivector = wss_multivector + pressure_multivector + pos_multivector + face_multivector + inlet_multivector
      sin_multivectors.append(total_multivector)

    except:
      damaged_sin_samples.append(sin_sample_idx)



100%|██████████| 1999/1999 [00:00<00:00, 37577.04it/s]


In [ ]:
# to save the sin_multivectors array in a pickle file on gdrive

# Specify the path on Google Drive
sin_pickle_file_path = '/content/drive/MyDrive/Deep_Learning/Project/sin_multivectors.pkl'

# Save the array to a pickle file on Google Drive
with open(sin_pickle_file_path, 'wb') as pickle_file:
    pickle.dump(sin_multivectors, pickle_file)

In [ ]:
print('number of damaged sin samples:\n', len(damaged_sin_samples))

number of damaged sin samples:
 0


In [ ]:
print('number of correct sin samples:\n', len(sin_samples))

number of correct sin samples:
 1999


In [ ]:
print('number of bif multivectors:\n', len(sin_multivectors))

In [ ]:
print('dimension of each sin multivector:\n', max(sin_multivectors[1].grades())) # if 4,  then the dimension is correct